In [ ]:

# Install PyTorch Geometric and related packages.
# (Adjust the URL if you are using a specific CUDA version; this example is for CPU)
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.0.0+cpu.html

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

# Load the online Cora dataset (automatically downloaded from the internet)
dataset = Planetoid(root='/tmp/Cora', name='Cora')


# Define a simple two-layer GCN model
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):  # ✅ Corrected __init__
        super(GCN, self).__init__()  # ✅ Corrected super().__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training, p=0.5)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


# Use GPU if available, otherwise CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(dataset.num_node_features, hidden_dim=16, output_dim=dataset.num_classes).to(device)
data = dataset[0].to(device)

# Set up optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Training function
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

# Testing function: reports accuracy on train, validation, and test splits
def test():
    model.eval()
    logits = model(data)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

# Run training for a small number of epochs (20 for demonstration)
for epoch in range(1, 21):
    loss = train()
    if epoch % 5 == 0:
        train_acc, val_acc, test_acc = test()
        print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
Epoch: 05, Loss: 1.4006, Train Acc: 0.9500, Val Acc: 0.6800, Test Acc: 0.6510
Epoch: 10, Loss: 0.7907, Train Acc: 0.9857, Val Acc: 0.7900, Test Acc: 0.7910
Epoch: 15, Loss: 0.4126, Train Acc: 0.9857, Val Acc: 0.7800, Test Acc: 0.7930
Epoch: 20, Loss: 0.2362, Train Acc: 1.0000, Val Acc: 0.7720, Test Acc: 0.7820
